In [14]:
import re

import pandas as pd
from pdfminer.high_level import extract_text

In [15]:
converted_pdf = extract_text("salaries.pdf")

with open("salaries.txt", "w") as out_file:
    out_file.write(converted_pdf)

In [16]:
# used google bard for regex patterns
def parse_employee_block(block):
    patterns = {
        "Name": "Name: (.+?)\\s{2,}",
        "First Hired": "First Hired: (.+?)\\n",
        "Home Orgn": "Home Orgn: (.+?)\\s{2,}",
        "Adj Service Date": "Adj Service Date: (.+?)\\n",
        "Job Orgn": "Job Orgn: (.+?)\\s{2,}",
        "Job Type": "Job Type: (.+?)\\n",
        "Job Title": "Job Title: (.+?)\\s{2,}",
        "Posn-Suff": "Posn-Suff: (.+?)\\n",
        "Rank": "Rank: (.+?)\\s{2,}",
        "Rank Effective Date": "Rank Effective Date: (.+?)\\n",
        "Appt Begin Date": "Appt Begin Date: (.+?)\\s{2,}",
        "Appt Percent": "Appt Percent: (.+?)\\n",
        "Appt End Date": "Appt End Date: (.+?)\\s{2,}",
        "Annual Salary Rate": "Annual Salary Rate:\\s+(.+?)\\s{2,}",
    }

    employee_data = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, block, re.DOTALL)
        employee_data[key] = match.group(1).strip() if match else None

    return employee_data


with open("salaries.txt", "r") as file:
    content = file.read()

# split the entires by horizontal bars and filter out empty strings, headers, and footers
employee_blocks = re.split(r"-{80,}", content)
employee_blocks = [
    block
    for block in employee_blocks
    if block.strip() and "Unclassified Personnel List" not in block
]

employees_data = [parse_employee_block(block) for block in employee_blocks]
faculty_df = pd.DataFrame(employees_data)

In [17]:
faculty_df.head()

,Name,First Hired,Home Orgn,Adj Service Date,Job Orgn,Job Type,Job Title,Posn-Suff,Rank,Rank Effective Date,Appt Begin Date,Appt Percent,Appt End Date,Annual Salary Rate
0,"Abakar, Reiman",01-AUG-2022,MSA - Dean of Students,01-AUG-2022,MSA - Dean of Students,P,OSU Assist Responder,C11439-00,No Rank,01-AUG-2022,01-AUG-2022,100,,65004.00
1,"Abbas, Houssam",31-DEC-2018,ESE - Sch Elect Engr/Comp Sci,31-DEC-2018,ESE - Sch Elect Engr/Comp Sci,P,Assistant Professor,C18336-00,Assistant Professor,31-DEC-2018,31-DEC-2018,100,,107901.00
2,"Abbasi, Bahman",01-AUG-2017,LCB - Acad Prog / Student Aff,01-AUG-2017,LCB - Acad Prog / Student Aff,P,Associate Professor,C11566-00,Associate Professor,16-SEP-2022,16-SEP-2022,100,,122499.00
3,"Abbott, Joanna",30-MAR-2015,XEM - Admissions,30-MAR-2015,XEM - Admissions,P,Asst Director of Articulation,C11138-00,No Rank,12-JUL-2021,12-JUL-2021,100,,57492.00
4,"Abel, Henry",01-JAN-1998,HHS - Hlth Hum Sci SW Outreach,01-JAN-1998,HHS - Hlth Hum Sci SW Outreach,P,Motorcycle Safety Instructor,C19996-00,No Rank,01-JAN-2013,01-JUL-2021,2,,55404.00


In [18]:
faculty_df.dtypes

Name                   object
First Hired            object
Home Orgn              object
Adj Service Date       object
Job Orgn               object
Job Type               object
Job Title              object
Posn-Suff              object
Rank                   object
Rank Effective Date    object
Appt Begin Date        object
Appt Percent           object
Appt End Date          object
Annual Salary Rate     object
dtype: object

In [19]:
faculty_df.to_csv("salaries.csv", index=False)
faculty_df = pd.read_csv("salaries.csv")
faculty_df.dtypes

Name                    object
First Hired             object
Home Orgn               object
Adj Service Date        object
Job Orgn                object
Job Type                object
Job Title               object
Posn-Suff               object
Rank                    object
Rank Effective Date     object
Appt Begin Date         object
Appt Percent             int64
Appt End Date           object
Annual Salary Rate     float64
dtype: object

In [20]:
faculty_df["Annual Salary Rate"].mean()

88299.85845447589

In [21]:
faculty_df["Annual Salary Rate"].median()

73728.0

In [47]:
unique_orgs_df = faculty_df["Job Orgn"].unique()

unique_orgs_dict = {}
for job in unique_orgs_df:
    unique_orgs_dict[job] = ""

In [48]:
unique_orgs_dict

{'MSA - Dean of Students': '',
 'ESE - Sch Elect Engr/Comp Sci': '',
 'LCB - Acad Prog / Student Aff': '',
 'XEM - Admissions': '',
 'HHS - Hlth Hum Sci SW Outreach': '',
 'CLA - Sch of Hist Phil & Rel': '',
 'ENG - College of Engineering': '',
 'MRS - Recreational Sports': '',
 'AFW - Fisheries and Wildlife': '',
 'EMM - Sch of Mech/Ind/Mfg Engr': '',
 'WHC - Univ Honors College': '',
 'YIA - Intercolleg Athletics': '',
 'KED - College of Education': '',
 'XUS - OSU Global Opportunities': '',
 'BFA - Sch of Fin/Acctg/BIS': '',
 'NCS - News&Research Communctns': '',
 'ATX - Enviro / Molecular Toxic': '',
 'AFC - Food Innovation Center': '',
 'TEX - Ext Jackson Co Office': '',
 'MSA - Coll Assistnc Migrnt Prg': '',
 'XUS - Academic Success &Engage': '',
 'ACS - Crop/Soil Sci Extension': '',
 'XEC - Ecampus': '',
 'QCU - Accts Payable & Travel': '',
 'QFA - Univ Human Rsrcs Central': '',
 'TEX - Ext Clackamas Co Office': '',
 'QCU - Financial Strategic Svcs': '',
 'XUS - Office of the Re

In [23]:
import json

with open("unique_orgs.json", "w") as out_file:
    json.dump(unique_orgs_dict, out_file)

used following prompt with google bard to classify departments/organizations:

> for the following key value pairs that show different departments of a university as keys and empty strings as vaalues, set the value of the string to: "a" if the department is administrative, "x" if it is an extension, "l" if related to liberal arts, "s" if related to science, "e" if related to engineering, "b" if related to business and finance, "m" if related to maintinence and unitilies, or "u" if there is not a clear category

with a very limited QC, could go back later for better accuracy

In [24]:
orgn_class = {
    "AAR - Applied Economics": "b",
    "ABE - Biol & Ecol Engineering": "e",
    "ABP - Ag Botany / Plant Path": "s",
    "ACB - Columbia Basin Exp Sta": "x",
    "ACS - Crop and Soil Science": "s",
    "ACS - Crop/Soil Sci Extension": "x",
    "ACT - Central Oregon Exp Sta": "x",
    "AEB - EOARC - Burns Exp Sta": "x",
    "AED - General Agriculture": "x",
    "AEU - EOARC - Union Exp Sta": "x",
    "AEU - Eastern Ore Univ Ag Prg": "x",
    "AFC - Food Innovation Center": "s",
    "AFM - Corvallis Farm Unit": "x",
    "AFS - Food Sci/Tech Extension": "x",
    "AFS - Food Science and Techno": "s",
    "AFW - Fisheries and Wildlife": "s",
    "AGA - College of Ag Admin": "a",
    "AGH - Ag Greenhouse Operation": "x",
    "AHC - Hemp Center": "x",
    "AHE - Hermiston Exp Sta": "x",
    "AHT - Horticulture": "s",
    "AHT - Horticulture Extension": "x",
    "AIP - Ag OR IPM Center": "x",
    "AKL - Klamath Basin Res&ExtCtr": "x",
    "AMA - Malheur Exp Sta": "x",
    "AMB - Microbiology (Ag)": "s",
    "AMC - Mid-Columbia Exp Sta": "x",
    "AMM - Marine Mammal Institute": "s",
    "ANW - North Willamette Exp Sta": "x",
    "ASC - Animal & Rnglnd Sciences": "s",
    "ASF - COMES - Newport Exp Sta": "x",
    "AST - Statistics (Ag)": "s",
    "ASU - Southern Oregon Exp Sta": "x",
    "ATX - Enviro / Molec Toxic Ext": "x",
    "ATX - Enviro / Molecular Toxic": "s",
    "BEM - Sch of Entr/Mgmt/Logstcs": "b",
    "BFA - Sch of Fin/Acctg/BIS": "b",
    "BMA - Sch of Mktg/Desn/Analytc": "b",
    "BUS - College of Business Dept": "b",
    "CLA - Art Department": "u",
    "CLA - Communication Studies": "l",
    "CLA - Ctr for the Humanities": "u",
    "CLA - Liberal Arts Admin": "a",
    "CLA - Sch Lang, Culture & Soc": "l",
    "CLA - Sch of Arts & Comm": "l",
    "CLA - Sch of Hist Phil & Rel": "l",
    "CLA - Sch of Psychological Sci": "l",
    "CLA - Sch of Wrtg Lit & Film": "l",
    "CLA - School of Public Policy": "l",
    "DAA - Faculty Affairs Admin": "a",
    "DLB - Library": "l",
    "ECB - Sch of Chem/Bio/Envr Eng": "e",
    "ECC - Sch of Civil/Constr Engr": "e",
    "EMM - Sch of Mech/Ind/Mfg Engr": "e",
    "ENG - College of Engineering": "e",
    "ENS - Sch Nuclear Sci & Engr": "e",
    "ESE - Sch Elect Engr/Comp Sci": "e",
    "FOR - College Forests": "m",
    "FOR - College of Forestry Adm": "a",
    "FOR - Ext Wildland Fire Progm": "x",
    "FOR - Forest Ecosyst & Society": "s",
    "FOR - Forest Eng/Resourcs/Mgmt": "e",
    "FOR - Forestry & Natrl Res Ext": "x",
    "FOR - TallWood DesignInstitute": "e",
    "FOR - Wood Science / Engr": "e",
    "GRD - Graduate School Admin": "a",
    "GRD - Water Res Graduate Prgm": "s",
    "HHS - Child Development Lab": "a",
    "HHS - EXT Fam/CommHlth OnCmps": "x",
    "HHS - Faculty/Staff Fitness": "m",
    "HHS - Hlth Hum Sci SW Outreach": "s",
    "HHS - Physical ActivityCourses": "m",
    "HHS - Public Hlth/HumanSci Adm": "a",
    "HHS - Sch of Bio/Pop Hlth Sci": "s",
    "HHS - Sch of Soc/Bhav Hlth Sci": "s",
    "INT - INTO OSU Program": "a",
    "JIS - Academic Technologies": "a",
    "JIS - Admin Technologies": "a",
    "JIS - Business Architecture": "a",
    "JIS - Customer Experience": "a",
    "JIS - Digital Exp, IT Dev+Eng": "a",
    "JIS - IT Infrastructure Svcs": "m",
    "JIS - Inst Analytcs & Reportng": "a",
    "JIS - Tech & Sol’ns Architectr": "a",
    "JIS - Telecommunication": "m",
    "JIS - Univ Info & Tech Admin": "a",
    "KED - College of Education": "l",
    "LCB - Acad Prog / Student Aff": "l",
    "LCB - Cascades Exec Office": "a",
    "LCB - Community Relatns/Admin": "a",
    "LCH - Housing & Dining Srvcs": "m",
    "MGV - ASOSU": "u",
    "MHD - Dining Centers": "m",
    "MHD - Univ Housing and Dining": "m",
    "MRS - Recreational Sports": "u",
    "MSA - Acad Svcs Stdnt Athletes": "a",
    "MSA - Academic Success Center": "a",
    "MSA - Basic Needs Center": "a",
    "MSA - Career Development Ctr": "a",
    "MSA - Coll Assistnc Migrnt Prg": "a",
    "MSA - Counseling Center": "a",
    "MSA - CrossCampusStratInttves": "a",
    "MSA - Ctr Fratrn/Sorority Life": "a",
    "MSA - Dean of Students": "a",
    "MSA - Disability Access Srvcs": "a",
    "MSA - Educ Opportunities Progm": "a",
    "MSA - Family Resource Center": "a",
    "MSA - International Services": "a",
    "MSA - NewStdntPrgs&FamlyOutrch": "a",
    "MSA - Student Conduct": "a",
    "MSA - TRiO/Student Supprt Svcs": "a",
    "MSA - Univ Exploratory Studies": "a",
    "MSA - Vice Prov / Student Aff": "a",
    "MSE - Comm Engagemnt & Ldrship": "u",
    "MSE - Craft Center": "u",
    "MSE - Diversity/Cultural Engag": "a",
    "MSE - Exprientl Learning/Activ": "u",
    "MSE - Stdnt Exprience/Engagemt": "a",
    "MSH - Student Health Services": "a",
    "MUN - Memorial Union": "u",
    "NCS - News&Research Communctns": "a",
    "NIA - Oregon State Productions": "u",
    "NIA - University Events": "a",
    "NIA - University Marketing": "a",
    "NIA - VP Univ Relations & Mktg": "a",
    "NLS - Conference Services": "m",
    "NPM - Printing & Mailing Svcs": "m",
    "OAS - Earth, Ocean & Atmo Sci": "s",
    "OAS - Ship Operations": "m",
    "PHR - Pharmacy Professnl Instr": "s",
    "QCP - Capital Planning & Devlp": "a",
    "QCP - Procur,Cntrcts&MatlsMmgt": "a",
    "QCU - Accts Payable & Travel": "a",
    "QCU - Central Payroll Office": "a",
    "QCU - Controller’s Office Admn": "a",
    "QCU - Financial Strategic Svcs": "a",
    "QCU - Treasury Operations": "a",
    "QCU - University Business Svcs": "a",
    "QFA - Budget/Resource Planning": "a",
    "QFA - F&A Information & Tech": "a",
    "QFA - Sr Assoc VP Fin & Admin": "a",
    "QFA - Univ Human Rsrcs Central": "a",
    "QFA - V Pres Finance and Admin": "a",
    "QFS - Facilities Ops & Maint": "m",
    "QIA - UnivFacInfra&OpsAdminSvc": "a",
    "QRS - Env Health & Safety": "s",
    "QRS - Insrnce & Risk Mgmt Svcs": "a",
    "QTS - Transportation Services": "m",
    "QUB - Dept of Public Safety": "m",
    "QUB - OSU Police Dept": "m",
    "RDR - AdvTech&MftgInst (ATAMI)": "e",
    "RDR - Coop Inst Marine Ecosyst": "s",
    "RDR - Ctr Excellnce Genome Res": "s",
    "RDR - Genome Research/Biocomp": "s",
    "RDR - Lab Animal": "s",
    "RDR - Linus Pauling Institute": "s",
    "RDR - STEM Research Center": "s",
    "RDR - Water/Watershed Institut": "s",
    "RIP - Radiation Center": "s",
    "RIP - VP for Research": "a",
    "RMS - Hatfield Marine Sci Ctr": "s",
    "RNR - INR-Or Biodvrsty InfoCtr": "s",
    "RNR - Institute Natrl Res Dir": "s",
    "RSG - Sea Grant": "s",
    "RSG - Sea Grant Extension": "x",
    "SBB - Biochem / Biophysics": "s",
    "SCH - Chemistry": "s",
    "SCI - College of Science Admin": "a",
    "SCI - NMR Facility": "s",
    "SMB - Microbiology (Science)": "s",
    "SMT - Mathematics": "s",
    "SPH - Physics": "s",
    "SST - Statistics (Science)": "s",
    "SZO - Integrative Biology": "s",
    "TEE - Extension & Engagement": "x",
    "TEE - KidSpirit": "a",
    "TEE - Open Campus": "x",
    "TEE - Outdoor Recreation Econ": "x",
    "TEE - Professional & Cont Educ": "a",
    "TEX - 4-H OR Conf/Educ Center": "x",
    "TEX - EXT Urban Outreach": "x",
    "TEX - Ext 4-H Yth Dev OffCmpus": "x",
    "TEX - Ext 4-H Yth Dev On-Cmpus": "x",
    "TEX - Ext Baker County Office": "x",
    "TEX - Ext Benton County Office": "x",
    "TEX - Ext Clackamas Co Office": "x",
    "TEX - Ext Clatsop Co Office": "x",
    "TEX - Ext Columbia Co Office": "x",
    "TEX - Ext Computer Svc Admin": "x",
    "TEX - Ext Coos County Office": "x",
    "TEX - Ext Crook County Office": "x",
    "TEX - Ext Curry County Office": "x",
    "TEX - Ext Deschutes Co Office": "x",
    "TEX - Ext Douglas County Offc": "x",
    "TEX - Ext Grant County Office": "x",
    "TEX - Ext Harney County Office": "x",
    "TEX - Ext Hood River Co Office": "x",
    "TEX - Ext Jackson Co Office": "x",
    "TEX - Ext Jefferson Co Office": "x",
    "TEX - Ext Josephine Co Office": "x",
    "TEX - Ext Klamath Co Office": "x",
    "TEX - Ext Lake County Office": "x",
    "TEX - Ext Lane County Office": "x",
    "TEX - Ext Lincoln Co Office": "x",
    "TEX - Ext Linn County Office": "x",
    "TEX - Ext Malheur Co Office": "x",
    "TEX - Ext Marion County Office": "x",
    "TEX - Ext Morrow County Office": "x",
    "TEX - Ext No Willamette Co Off": "x",
    "TEX - Ext Polk County Office": "x",
    "TEX - Ext Sherman Co Office": "x",
    "TEX - Ext Tillamook Co Office": "x",
    "TEX - Ext Umatilla - Hermiston": "x",
    "TEX - Ext Umatilla Co Office": "x",
    "TEX - Ext Union County Office": "x",
    "TEX - Ext Wallowa Co Office": "x",
    "TEX - Ext Warm Springs": "x",
    "TEX - Ext Wasco County Office": "x",
    "TEX - Ext Washington Co Office": "x",
    "TEX - Ext Wheeler Co Office": "x",
    "TEX - Ext Yamhill Co Office": "x",
    "TEX - Extension Communications": "x",
    "TEX - Extension Service Admin": "x",
    "TOS - Outdoor School": "m",
    "UPR - Audit, Risk & Compliance": "a",
    "UPR - Equal Opportunity&Access": "a",
    "UPR - General Counsel Office": "a",
    "UPR - Government Relations": "a",
    "UPR - Institutional Diversity": "a",
    "UPR - OSU Board of Trustees": "a",
    "UPR - Office of the President": "a",
    "UPR - University Ombuds Office": "a",
    "VBS - Vet Biomedical Science": "s",
    "VCS - Vet Clinical Sciences": "s",
    "VDL - Vet Diagnostic Lab": "s",
    "VMD - Veterinary Medicine": "s",
    "VTH - Vet Teaching Hospital": "s",
    "WHC - Univ Honors College": "l",
    "XEC - Ecampus": "a",
    "XEM - Admissions": "a",
    "XEM - Enrollment Mgmt InfoTech": "a",
    "XEM - Financial Aid": "a",
    "XEM - Gear Up Program": "a",
    "XEM - Office of Scholarships": "a",
    "XEM - Pre-College Programs": "a",
    "XEM - The SMILE Program": "a",
    "XEM - Vice Prov Enrollmnt Mgmt": "a",
    "XMS - Marine Studies Initiatv": "s",
    "XPV - Provost / Exec Vice Pres": "a",
    "XUS - Academic Success &Engage": "a",
    "XUS - Assessment & Accreditatn": "a",
    "XUS - OSU Global Opportunities": "a",
    "XUS - Office of the Registrar": "a",
    "XUS - VP Academic Affairs": "a",
    "YIA - Intercolleg Athletics": "u",
    "ZSS - Univ Shared Svcs Entrprs": "a",
}

In [25]:
faculty_df.columns

Index(['Name', 'First Hired', 'Home Orgn', 'Adj Service Date', 'Job Orgn',
       'Job Type', 'Job Title', 'Posn-Suff', 'Rank', 'Rank Effective Date',
       'Appt Begin Date', 'Appt Percent', 'Appt End Date',
       'Annual Salary Rate'],
      dtype='object')

In [26]:
faculty_df["orgn_class"] = faculty_df["Job Orgn"].map(orgn_class)

In [27]:
faculty_df.head()

,Name,First Hired,Home Orgn,Adj Service Date,Job Orgn,Job Type,Job Title,Posn-Suff,Rank,Rank Effective Date,Appt Begin Date,Appt Percent,Appt End Date,Annual Salary Rate,orgn_class
0,"Abakar, Reiman",01-AUG-2022,MSA - Dean of Students,01-AUG-2022,MSA - Dean of Students,P,OSU Assist Responder,C11439-00,No Rank,01-AUG-2022,01-AUG-2022,100,NaN,65004.0,a
1,"Abbas, Houssam",31-DEC-2018,ESE - Sch Elect Engr/Comp Sci,31-DEC-2018,ESE - Sch Elect Engr/Comp Sci,P,Assistant Professor,C18336-00,Assistant Professor,31-DEC-2018,31-DEC-2018,100,NaN,107901.0,e
2,"Abbasi, Bahman",01-AUG-2017,LCB - Acad Prog / Student Aff,01-AUG-2017,LCB - Acad Prog / Student Aff,P,Associate Professor,C11566-00,Associate Professor,16-SEP-2022,16-SEP-2022,100,NaN,122499.0,l
3,"Abbott, Joanna",30-MAR-2015,XEM - Admissions,30-MAR-2015,XEM - Admissions,P,Asst Director of Articulation,C11138-00,No Rank,12-JUL-2021,12-JUL-2021,100,NaN,57492.0,a
4,"Abel, Henry",01-JAN-1998,HHS - Hlth Hum Sci SW Outreach,01-JAN-1998,HHS - Hlth Hum Sci SW Outreach,P,Motorcycle Safety Instructor,C19996-00,No Rank,01-JAN-2013,01-JUL-2021,2,NaN,55404.0,s


In [28]:
faculty_df["orgn_class"].value_counts()

orgn_class
s    1176
a     826
l     626
e     418
x     342
m     205
u     165
b     163
Name: count, dtype: int64

In [29]:
assistant_prof_df = faculty_df[faculty_df["Rank"] == "Assistant Professor"]
assistant_prof_df.head()

,Name,First Hired,Home Orgn,Adj Service Date,Job Orgn,Job Type,Job Title,Posn-Suff,Rank,Rank Effective Date,Appt Begin Date,Appt Percent,Appt End Date,Annual Salary Rate,orgn_class
1,"Abbas, Houssam",31-DEC-2018,ESE - Sch Elect Engr/Comp Sci,31-DEC-2018,ESE - Sch Elect Engr/Comp Sci,P,Assistant Professor,C18336-00,Assistant Professor,31-DEC-2018,31-DEC-2018,100,NaN,107901.0,e
25,"Agor, Joseph Kapena",18-FEB-2019,EMM - Sch of Mech/Ind/Mfg Engr,18-FEB-2019,EMM - Sch of Mech/Ind/Mfg Engr,P,Assistant Professor,C15597-00,Assistant Professor,16-SEP-2019,16-SEP-2019,100,NaN,99036.0,e
37,"Akula, Pavan",09-JUN-2020,ECC - Sch of Civil/Constr Engr,16-DEC-2021,ECC - Sch of Civil/Constr Engr,P,Assistant Professor,C11678-00,Assistant Professor,16-DEC-2021,16-DEC-2021,100,NaN,98568.0,e
42,"Albert, Loren",01-JUL-2022,FOR - Forest Ecosyst & Society,30-DEC-2022,FOR - Forest Ecosyst & Society,P,Assistant Professor,C15467-00,Assistant Professor,30-DEC-2022,30-DEC-2022,100,NaN,92007.0,s
132,"Arocho, Ingrid",31-DEC-2014,ECC - Sch of Civil/Constr Engr,31-DEC-2014,ECC - Sch of Civil/Constr Engr,P,Assistant Professor,C14850-00,Assistant Professor,31-DEC-2014,31-DEC-2014,100,NaN,99522.0,e


In [30]:
assistant_prof_df.shape

(160, 15)

In [31]:
assistant_prof_df["orgn_class"].value_counts()

orgn_class
s    59
e    44
l    30
b    15
x    12
Name: count, dtype: int64

In [32]:
grouped_class_ds = assistant_prof_df.groupby("orgn_class")["Annual Salary Rate"].mean()
grouped_class_count = assistant_prof_df.groupby("orgn_class")["Annual Salary Rate"].count().rename("count")

result_df = pd.concat([grouped_class_ds, grouped_class_count], axis=1)
result_df = result_df.sort_values("Annual Salary Rate", ascending=False)
result_df

,Annual Salary Rate,count
orgn_class,,
b,147961.800000,15
e,103177.636364,44
s,95206.067797,59
x,87148.000000,12
l,75880.700000,30


In [33]:
def get_job_type_mean_salary_by_class(df, rank):
    df = df[df["Rank"] == rank]
    grouped_class_ds = df.groupby("orgn_class")["Annual Salary Rate"].mean()
    grouped_class_count = df.groupby("orgn_class")["Annual Salary Rate"].count().rename("count")
    result_df = pd.concat([grouped_class_ds, grouped_class_count], axis=1)
    result_df = result_df.sort_values("Annual Salary Rate", ascending=False)

    return result_df

In [34]:
instructor_df = get_job_type_mean_salary_by_class(faculty_df, "Instructor")
instructor_df

,Annual Salary Rate,count
orgn_class,,
e,74481.545455,44
b,68635.323529,34
s,65197.867347,98
a,53111.500000,6
l,50145.508651,289
x,49913.142857,7
u,45000.000000,1
m,35167.500000,2


In [35]:
associate_df = get_job_type_mean_salary_by_class(faculty_df, "Associate Professor")
associate_df

,Annual Salary Rate,count
orgn_class,,
b,161896.250000,24
e,124957.555556,54
s,110145.137615,109
x,100563.428571,28
l,93249.846154,78


In [43]:
bot_df = faculty_df[faculty_df["Job Orgn"] == "ABP - Ag Botany / Plant Path"]
bot_df

,Name,First Hired,Home Orgn,Adj Service Date,Job Orgn,Job Type,Job Title,Posn-Suff,Rank,Rank Effective Date,Appt Begin Date,Appt Percent,Appt End Date,Annual Salary Rate,orgn_class
91,"Anderson, Jeff",01-DEC-2014,ABP - Ag Botany / Plant Path,29-MAY-2015,ABP - Ag Botany / Plant Path,P,Associate Professor,C14617-00,Associate Professor,16-SEP-2021,29-MAY-2015,100,NaN,104472.0,s
186,"Baldino, Katelyn Dane",08-DEC-2020,ABP - Ag Botany / Plant Path,27-MAR-2023,ABP - Ag Botany / Plant Path,P,Faculty Research Assistant,C10728-00,Faculty Research Assistant,27-MAR-2023,27-MAR-2023,100,30-JUN-2024,51504.0,s
252,"Bearden, Nathan James",03-JAN-2016,ABP - Ag Botany / Plant Path,03-JAN-2016,ABP - Ag Botany / Plant Path,P,Faculty Research Assistant,C10833-00,Faculty Research Assistant,03-FEB-2022,01-JUL-2022,100,30-JUN-2023,52020.0,s
272,"Behrenfeld, Michael Joseph",01-AUG-2004,ABP - Ag Botany / Plant Path,01-AUG-2004,ABP - Ag Botany / Plant Path,P,Professor,C18092-00,Professor,16-SEP-2008,16-SEP-2008,100,NaN,161271.0,s
513,"Busby, Posy E",04-JUN-2001,ABP - Ag Botany / Plant Path,04-JUN-2001,ABP - Ag Botany / Plant Path,P,Associate Professor,C16138-00,Associate Professor,16-SEP-2022,16-SEP-2016,100,NaN,102960.0,s
737,"Cooper, Laurel",01-FEB-1997,ABP - Ag Botany / Plant Path,28-SEP-2001,ABP - Ag Botany / Plant Path,P,Research Associate,C18259-00,Research Associate,01-NOV-2009,01-JUL-2021,50,30-JUN-2023,56412.0,s
796,"Curtis, Marc James",30-SEP-1996,ABP - Ag Botany / Plant Path,30-SEP-1996,ABP - Ag Botany / Plant Path,P,Senior Instructor I,C17648-00,Senior Instructor I,01-JUL-2022,01-JUL-2022,100,30-JUN-2024,85164.0,s
927,"Dolja, Valerian V",29-MAR-1994,ABP - Ag Botany / Plant Path,29-MAR-1994,ABP - Ag Botany / Plant Path,P,Professor,C10662-00,Professor,01-JUL-2001,01-JUL-1998,75,NaN,204432.0,s
1171,"Fowler Jr, John E",16-SEP-1997,ABP - Ag Botany / Plant Path,16-SEP-1997,ABP - Ag Botany / Plant Path,P,Professor,C10763-00,Professor,01-JUL-2016,01-JUL-1998,100,NaN,131940.0,s
1337,"Goyer, Aymeric J",15-OCT-2005,ABP - Ag Botany / Plant Path,15-OCT-2005,ABP - Ag Botany / Plant Path,P,Associate Professor (Sr Res),C17600-00,"Associate Professor, Sr Res",01-JUL-2017,01-JUL-2022,100,30-JUN-2024,99936.0,s
